In [2]:
import pandas as pd
import re
from pathlib import Path

# Load the uploaded Excel file to inspect its structure
__PATH__ = Path().parent
__DATA__ = __PATH__ / 'db'
data = pd.ExcelFile(__DATA__ / 'hasil hasil belajar.xlsx')

# Display sheet names and preview first few rows of each sheet
sheet_names = data.sheet_names
sheets_preview = {sheet: data.parse(sheet).head() for sheet in sheet_names}

sheet_names, sheets_preview

import numpy as np

# Function to convert time strings to total seconds
def time_to_seconds(time_str):
    minutes, seconds = map(int, time_str.replace('detik', '').split('menit,'))
    return minutes * 60 + seconds

# Load data from sheets
sheet1 = data.parse('Sheet1')
sheet2 = data.parse('Sheet2')
rules = data.parse('Sheet3')

# Combine Sheet1 and Sheet2 for uniform processing
combined_data = pd.concat([sheet1, sheet2], ignore_index=True)

# Process the 'lama_berlatih' column (convert time to seconds)
combined_data['lama_berlatih (detik)'] = combined_data['lama_berlatih (min 5, mak 15)'].apply(time_to_seconds)

# Fuzzy membership functions
def fuzzy_rata2_akses(value):
    if value <= 400:
        return "kecil"
    elif 400 < value <= 600:
        return "sedang"
    else:
        return "besar"

def fuzzy_lama_berlatih(value):
    if value <= 300:
        return "kecil"
    elif 300 < value <= 900:
        return "sedang"
    else:
        return "besar"

def fuzzy_nilai(value):
    if value <= 50:
        return "kecil"
    elif 50 < value <= 75:
        return "sedang"
    else:
        return "besar"

# Apply fuzzification
combined_data['rata2 Lama akses materi (fuzzy)'] = combined_data['rata2 Lama akses materi'].apply(fuzzy_rata2_akses)
combined_data['lama_berlatih (fuzzy)'] = combined_data['lama_berlatih (detik)'].apply(fuzzy_lama_berlatih)
combined_data['nilai (fuzzy)'] = combined_data['nilai'].apply(fuzzy_nilai)

# Preview processed data
combined_data.head()

# Match fuzzy data with rules to calculate CF
def calculate_cf(row, rules):
    for _, rule in rules.iterrows():
        if (
            row['rata2 Lama akses materi (fuzzy)'] == rule['rata2 Lama akses materi']
            and row['lama_berlatih (fuzzy)'] == rule['lama_berlatih (min 5, mak 15)']
            and row['nilai (fuzzy)'] == rule['nilai']
        ):
            return rule['kepercayaan']
    return 0  # Default CF if no matching rule

# Apply CF calculation
combined_data['calculated_CF'] = combined_data.apply(calculate_cf, axis=1, rules=rules)

# Preview data with calculated CF
combined_data[['no', 'rata2 Lama akses materi', 'lama_berlatih (detik)', 'nilai',
                'rata2 Lama akses materi (fuzzy)', 'lama_berlatih (fuzzy)', 'nilai (fuzzy)',
                'keyakinan', 'calculated_CF']]

# Rename columns
df_sheet1 =sheet1.rename(columns={'keyakinan': 'certainty_factor'})
df_sheet2 = sheet2.rename(columns={'keyakinan': 'certainty_factor'})
df_sheet3 = rules.rename(columns={'kepercayaan': 'certainty_factor'})

# Convert `lama_berlatih (min 5, mak 15)` to minutes as float
def convert_to_minutes(time_str):
    minutes, seconds = map(int, re.findall(r'\d+', time_str))
    return minutes + seconds / 60

df_sheet1['lama_berlatih (min 5, mak 15)'] = df_sheet1['lama_berlatih (min 5, mak 15)'].apply(convert_to_minutes)
df_sheet2['lama_berlatih (min 5, mak 15)'] = df_sheet2['lama_berlatih (min 5, mak 15)'].apply(convert_to_minutes)

# Drop unnecessary columns
df_sheet1 = df_sheet1.drop(['kesimpulannya', 'no'], axis=1)
df_sheet2 = df_sheet2.drop(['kesimpulannya', 'no'], axis=1)
df_sheet3 = df_sheet3.drop(['Unnamed: 0'], axis=1)

# Define the membership function
def calculate_membership(x, points):
    if x <= points[0] or x >= points[3]:
        return 0
    elif points[0] < x <= points[1]:
        return (x - points[0]) / (points[1] - points[0])
    elif points[1] < x <= points[2]:
        return 1
    elif points[2] < x < points[3]:
        return (points[3] - x) / (points[3] - points[2])

# Apply the function to calculate fuzzy values for the specified columns
for df in [df_sheet1, df_sheet2]:
    df['rata2 Lama akses materi'] = df['rata2 Lama akses materi'].apply(lambda x: calculate_membership(x, [300, 450, 550, 700]))
    df['lama_berlatih (min 5, mak 15)'] = df['lama_berlatih (min 5, mak 15)'].apply(lambda x: calculate_membership(x, [5, 10, 20, 25]))
    df['nilai'] = df['nilai'].apply(lambda x: calculate_membership(x, [0, 40, 70, 100]))

# Concatenate the datasets
df_combined = pd.concat([df_sheet1, df_sheet2], ignore_index=True)

# Define the fuzzy logic system function
def fuzzy_logic_system(rata2_lama_akses_materi, lama_berlatih, nilai):
    max_cf = -1
    result = None

    for _, row in df_sheet3.iterrows():
        cf_rata2_lama_akses_materi = calculate_membership(rata2_lama_akses_materi, [300, 450, 550, 700])
        cf_lama_berlatih = calculate_membership(lama_berlatih, [5, 10, 20, 25])
        cf_nilai = calculate_membership(nilai, [0, 40, 70, 100])

        cf = min(cf_rata2_lama_akses_materi, cf_lama_berlatih, cf_nilai)
        if cf > max_cf:
            max_cf = cf
            result = row['lulus']

    return result, max_cf

# Apply the function to the combined DataFrame
df_combined[['lulus', 'certainty_factor_lulus']] = df_combined.apply(
    lambda row: fuzzy_logic_system(row['rata2 Lama akses materi'], row['lama_berlatih (min 5, mak 15)'], row['nilai']),
    axis=1,
    result_type='expand'
)

# Defuzzification
def defuzzify(row):
    # Tentukan bobot berdasarkan kesimpulan
    if row['lulus'] == 'lulus':
        bobot_lulus = row['certainty_factor_lulus']
        bobot_tidak_lulus = 1 - bobot_lulus
    else:
        bobot_tidak_lulus = row['certainty_factor_lulus']
        bobot_lulus = 1 - bobot_tidak_lulus

    # Defuzzifikasi dengan metode centroid
    nilai_crisp = (bobot_lulus * 100 + bobot_tidak_lulus * 0) / (bobot_lulus + bobot_tidak_lulus)
    return nilai_crisp

# Terapkan defuzzifikasi pada DataFrame
df_combined['nilai_crisp'] = df_combined.apply(defuzzify, axis=1)

# Tampilkan hasil defuzzifikasi
print(df_combined[['rata2 Lama akses materi', 'lama_berlatih (min 5, mak 15)', 'nilai', 'lulus', 'certainty_factor_lulus', 'nilai_crisp']].head().to_markdown(index=False, numalign="left", stralign="left"))

| rata2 Lama akses materi   | lama_berlatih (min 5, mak 15)   | nilai    | lulus       | certainty_factor_lulus   | nilai_crisp   |
|:--------------------------|:--------------------------------|:---------|:------------|:-------------------------|:--------------|
| 0.733333                  | 0.506667                        | 0.833333 | tidak lulus | 0                        | 100           |
| 1                         | 1                               | 0.766667 | tidak lulus | 0                        | 100           |
| 1                         | 0.646667                        | 0.740667 | tidak lulus | 0                        | 100           |
| 0.766667                  | 0                               | 0.740667 | tidak lulus | 0                        | 100           |
| 0.693333                  | 1                               | 0.740667 | tidak lulus | 0                        | 100           |
